# HEA Model Training & Analysis

**核心内容**:
1. 数据加载与探索
2. **K-Fold交叉验证**（评估真实泛化能力）
3. 超参数优化（Optuna贝叶斯优化）
4. 模型训练与评估
5. **SHAP物理可解释性分析**（提取材料设计规律）
6. 模型保存与部署

## 1. 环境设置

In [ ]:
# 导入必要库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

import shap
import optuna
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

%matplotlib inline

print("✓ 环境设置完成")

## 2. 数据加载

In [ ]:
# 加载GBFS精简后的特征集
df = pd.read_csv('d:/ML/HEAC 0.2/datasets/hea_selected_features.csv')

print(f"数据形状: {df.shape}")
print(f"\n可用列:\n{df.columns.tolist()}")

# 显示前几行
df.head()

In [ ]:
# 数据统计
df.describe()

## 3. 数据准备

In [ ]:
# 定义目标变量（根据需要修改）
TARGET = 'HV, kgf/mm2'  # 可选: 'TRS, MPa', 'KIC, MPa·m1/2'

# 分离特征和目标
target_candidates = ['HV, kgf/mm2', 'TRS, MPa', 'KIC, MPa·m1/2']
feature_cols = [c for c in df.columns if c not in target_candidates]

X = df[feature_cols].copy()
y = df[TARGET].copy()

# 处理缺失值
y = pd.to_numeric(y, errors='coerce')
valid_idx = y.notna()

X = X[valid_idx]
y = y[valid_idx]

print(f"有效样本数: {len(X)}")
print(f"特征数: {len(feature_cols)}")

In [ ]:
# Train/Test分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 转回DataFrame保留列名
X_train = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print("✓ 数据标准化完成")

## 4. K-Fold交叉验证基线模型

> ⚠️ **重要**: 因为数据量有限，必须使用K-Fold交叉验证来评估模型的真实泛化能力

In [ ]:
# 使用默认参数的XGBoost作为基线
baseline_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.1,
    random_state=42
)

# 5-Fold交叉验证
cv_scores = cross_val_score(
    baseline_model,
    X_train,
    y_train,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

print(f"5-Fold CV R² Scores: {cv_scores}")
print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

## 5. Optuna超参数优化 + K-Fold CV

In [ ]:
def objective(trial):
    """Optuna目标函数"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42
    }
    
    model = xgb.XGBRegressor(**params)
    
    # K-Fold交叉验证
    cv_scores = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='r2',
        n_jobs=-1
    )
    
    return cv_scores.mean()

# 创建study并优化
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)

print("开始Optuna优化（这可能需要几分钟）...")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print(f"\n最佳CV R²: {study.best_value:.4f}")
print(f"\n最佳参数:\n{study.best_params}")

In [ ]:
# 可视化优化历史
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 优化历史
trials_df = study.trials_dataframe()
ax = axes[0]
ax.plot(trials_df['number'], trials_df['value'], 'b-', alpha=0.3, label='Trial Score')
ax.plot(trials_df['number'], trials_df['value'].cummax(), 'r-', linewidth=2, label='Best Score')
ax.set_xlabel('Trial Number')
ax.set_ylabel('CV R²')
ax.set_title('Optimization History')
ax.legend()
ax.grid(True, alpha=0.3)

# 参数重要性
importances = optuna.importance.get_param_importances(study)
importance_df = pd.DataFrame({
    'Parameter': list(importances.keys()),
    'Importance': list(importances.values())
}).sort_values('Importance', ascending=True)

ax = axes[1]
ax.barh(importance_df['Parameter'], importance_df['Importance'])
ax.set_xlabel('Importance')
ax.set_title('Hyperparameter Importance')

plt.tight_layout()
plt.show()

## 6. 训练最终模型

In [ ]:
# 使用最佳参数训练模型
best_model = xgb.XGBRegressor(**study.best_params)

best_model.fit(
    X_train, y_train,
    eval_set=[X_test, y_test)],
    early_stopping_rounds=50,
    verbose=False
)

# 预测
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# 计算指标
metrics = pd.DataFrame({
    'Dataset': ['Train', 'Test'],
    'R²': [
        r2_score(y_train, y_pred_train),
        r2_score(y_test, y_pred_test)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(y_train, y_pred_train)),
        np.sqrt(mean_squared_error(y_test, y_pred_test))
    ],
    'MAE': [
        mean_absolute_error(y_train, y_pred_train),
        mean_absolute_error(y_test, y_pred_test)
    ]
})

print("模型性能:")
print(metrics)

In [ ]:
# 可视化预测结果
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 预测vs实际
ax = axes[0]
ax.scatter(y_test, y_pred_test, alpha=0.6, s=50)
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title(f'Prediction vs Actual (Test R²={metrics.loc[1, "R²"]:.4f})')
ax.grid(True, alpha=0.3)

# 残差图
ax = axes[1]
residuals = y_test - y_pred_test
ax.scatter(y_pred_test, residuals, alpha=0.6, s=50)
ax.axhline(y=0, color='r', linestyle='--', lw=2)
ax.set_xlabel('Predicted')
ax.set_ylabel('Residuals')
ax.set_title('Residual Plot')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. SHAP物理可解释性分析

> 🎯 **关键步骤**: 提取物理规律，为材料设计提供指导
>
> 例如: "提高同系温度 (Ratio_MeltingT) 到底是让硬度变高还是变低?"

In [ ]:
# 创建SHAP explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

print("✓ SHAP值计算完成")

In [ ]:
# Summary Bar Plot（特征重要性）
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.tight_layout()
plt.show()

In [ ]:
# Beeswarm Plot（物理规律核心可视化）
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test, max_display=20)
plt.tight_layout()
plt.show()

### 自动提取物理规律

In [ ]:
# 提取Top特征的物理影响
shap_importance = np.abs(shap_values).mean(axis=0)
top_features_idx = np.argsort(shap_importance)[-10:][::-1]

insights = []
for idx in top_features_idx:
    feat_name = X_test.columns[idx]
    feat_shap = shap_values[:, idx]
    feat_values = X_test.iloc[:, idx].values
    
    # 计算相关性（判断正负影响）
    correlation = np.corrcoef(feat_values, feat_shap)[0, 1]
    
    direction = "正相关" if correlation > 0 else "负相关"
    effect = "增加" if correlation > 0 else "降低"
    importance = shap_importance[idx]
    
    insights.append({
        '特征': feat_name,
        '重要性': f"{importance:.4f}",
        '影响方向': direction,
        f'对{TARGET}的影响': f"特征值↑ → 预测{effect}"
    })

insights_df = pd.DataFrame(insights)
print("\n物理规律提取:")
insights_df

In [ ]:
# Dependence Plot（详细查看关键特征）
# 选择Top 3特征展示
top_3_features = X_test.columns[top_features_idx[:3]]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, feat in enumerate(top_3_features):
    shap.dependence_plot(
        feat,
        shap_values,
        X_test,
        ax=axes[i],
        show=False
    )

plt.tight_layout()
plt.show()

## 8. 模型保存

In [ ]:
import joblib
import os

# 创建models目录
os.makedirs('models', exist_ok=True)

# 打包模型
model_package = {
    'model': best_model,
    'scaler': scaler,
    'feature_names': list(X_train.columns),
    'target_name': TARGET,
    'best_params': study.best_params,
    'cv_score': study.best_value,
    'metrics': metrics.to_dict(),
    'shap_explainer': explainer
}

# 保存
model_path = f"models/XGBoost_{TARGET.replace(', ', '_').replace('·', '').replace('/', '_')}.pkl"
joblib.dump(model_package, model_path)

print(f"✓ 模型已保存到: {model_path}")

## 9. 预测示例

In [ ]:
# 加载模型并预测新样本
loaded_package = joblib.load(model_path)

def predict_new_sample(sample_dict):
    """预测新样本"""
    X_new = pd.DataFrame([sample_dict])
    X_new = X_new[loaded_package['feature_names']]
    X_new = loaded_package['scaler'].transform(X_new)
    
    prediction = loaded_package['model'].predict(X_new)
    
    return prediction[0]

# 示例：使用测试集第一个样本
sample = X_test.iloc[0].to_dict()
prediction = predict_new_sample(sample)

print(f"预测值: {prediction:.2f}")
print(f"实际值: {y_test.iloc[0]:.2f}")
print(f"误差: {abs(prediction - y_test.iloc[0]):.2f}")